In [1]:
 !git clone https://github.com/Trusted-AI/AIX360

Cloning into 'AIX360'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1374 (delta 29), reused 18 (delta 4), pack-reused 1304
Receiving objects: 100% (1374/1374), 357.25 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (761/761), done.


In [2]:
cd /content/AIX360

/content/AIX360


In [3]:
!pip install -e .

Obtaining file:///content/AIX360
     |████████████████████████████████| 109.2MB 86kB/s 
     |████████████████████████████████| 109.8MB 53kB/s 
     |████████████████████████████████| 276kB 44.0MB/s 
     |████████████████████████████████| 266kB 42.1MB/s 
     |████████████████████████████████| 491kB 41.4MB/s 
     |████████████████████████████████| 3.2MB 37.7MB/s 
  Created wheel for qpsolvers: filename=qpsolvers-1.4-cp36-none-any.whl size=18454 sha256=be15f85d8673763a99f6023ebe753bee6f246cb1a0edbd7e538d1d59e2e9f64b
  Stored in directory: /root/.cache/pip/wheels/ce/a7/5f/2af411a5eea3909096a303f84b7d6ff84e440081e8baccb4f2
  Created wheel for lime: filename=lime-0.1.1.37-cp36-none-any.whl size=284277 sha256=d269bd588f2ce8f9af0baa1bce530b4b7aae5e8b1d98b9ee7563331b57213b0e
  Stored in directory: /root/.cache/pip/wheels/c1/38/e7/50d75d4fb75afa604570dc42f20c5c5f5ab26d3fbe8d6ef27b
  Created wheel for shap: filename=shap-0.34.0-cp36-cp36m-linux_x86_64.whl size=383179 sha256=ca8ca39685c6988b5

In [4]:
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()
import pandas as pd
bc_df = pd.DataFrame(bc.data, columns=bc.feature_names)
bc_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
bc_df.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(bc_df, bc.target, test_size = 0.2, random_state = 31)
from aix360.algorithms.rbm import FeatureBinarizer
fb = FeatureBinarizer(negations=True)
X_train_fb = fb.fit_transform(X_train)
X_test_fb = fb.transform(X_test)
X_train_fb['mean radius'][:8]


operation     <=                              ...      >                            
value     10.254 11.328 11.942 12.604 13.270  ... 13.270 14.142 15.058 17.026 19.324
468            0      0      0      0      0  ...      1      1      1      1      0
179            0      0      0      0      1  ...      0      0      0      0      0
114            1      1      1      1      1  ...      0      0      0      0      0
35             0      0      0      0      0  ...      1      1      1      0      0
88             0      0      0      1      1  ...      0      0      0      0      0
507            0      1      1      1      1  ...      0      0      0      0      0
213            0      0      0      0      0  ...      1      1      1      1      0
134            0      0      0      0      0  ...      1      1      1      1      0

[8 rows x 18 columns]

In [7]:
from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG
boolean_model = BooleanRuleCG(silent=True)
explainer = BRCGExplainer(boolean_model)
explainer.fit(X_train_fb, Y_train)
Y_pred = explainer.predict(X_test_fb)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy = {accuracy_score(Y_test, Y_pred)}')
print(f'Precision = {precision_score(Y_test, Y_pred)}')
print(f'Recall = {recall_score(Y_test, Y_pred)}')
print(f'F1 = {f1_score(Y_test, Y_pred)}')
explainer.explain()

Accuracy = 0.9298245614035088
Precision = 0.9538461538461539
Recall = 0.9253731343283582
F1 = 0.9393939393939394


{'isCNF': False,
 'rules': ['compactness error > 0.01 AND worst concavity <= 0.22 AND worst symmetry <= 0.28',
  'mean texture <= 15.46 AND mean concavity <= 0.15 AND area error <= 54.16',
  'fractal dimension error > 0.00 AND worst area <= 680.60 AND worst concave points <= 0.18',
  'mean concave points <= 0.05 AND perimeter error <= 3.80 AND worst area <= 930.88 AND worst smoothness <= 0.16']}

In [16]:
e = explainer.explain()
isCNF = 'Predict Y=0 if ANY of the following rules are satisfied, otherwise Y=1:'
notCNF = 'Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:'
print(isCNF if e['isCNF'] else notCNF)
print()
for rule in e['rules']:
    print(f'  - {rule}')


Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:

  - compactness error > 0.01 AND worst concavity <= 0.22 AND worst symmetry <= 0.28
  - mean texture <= 15.46 AND mean concavity <= 0.15 AND area error <= 54.16
  - fractal dimension error > 0.00 AND worst area <= 680.60 AND worst concave points <= 0.18
  - mean concave points <= 0.05 AND perimeter error <= 3.80 AND worst area <= 930.88 AND worst smoothness <= 0.16


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from aix360.algorithms.protodash import ProtodashExplainer, get_Gaussian_Data

In [ ]:
data=bc_df[["mean radius","mean texture","mean area", "area error","worst area","worst concavity","concave points error"]]
data["target"]=bc.target
data.head()

In [14]:
data_numpy = data.to_numpy()
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(data_numpy,data_numpy, m=2, sigma=2)
inc_prototypes =data.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes

/usr/local/lib/python3.6/dist-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/usr/local/lib/python3.6/dist-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/usr/local/lib/python3.6/dist-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/usr/local/lib/python3.6/dist-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


,mean radius,mean texture,mean area,area error,worst area,worst concavity,concave points error,target,Weights of Prototypes
340,14.42,16.54,641.2,32.14,862.1,0.3755,0.01243,1,0.99
212,28.11,18.47,2499.0,525.60,2499.0,0.3201,0.01407,0,0.01


In [30]:
test=data[1:2]
test

,mean radius,mean texture,mean area,area error,worst area,worst concavity,concave points error,target
1,20.57,17.77,1326.0,74.08,1956.0,0.2416,0.0134,0


In [31]:
test_numpy=test.to_numpy()
explain1 =data.drop([1])
explain =data.drop([1]).to_numpy()
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(test_numpy,explain, m=1, sigma=2)
inc_prototypes =explain1.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes

,mean radius,mean texture,mean area,area error,worst area,worst concavity,concave points error,target,Weights of Prototypes
393,21.61,22.28,1407.0,80.99,2081.0,0.7053,0.01288,0,1.0
